## 使用 Selector 实现意图路由

智能体路由可以扩展为更广泛的场景，比如根据不同的工具集合、向量库类型等，配置为不同的智能体，
然后根据用户的意图和场景，使用 Selector 动态选择最佳的智能体。


Selector 有两个参数：
- condition 可以自定义，或者采用内置的方法名称，包括：
    - first : 第一个，这是默认选项，主要用于测试
    - random : 随机选一个
    - similar : 输入与备选 Runnable 的 description 属性接近（要使用这一方法必须提供向量嵌入模型）
- agents 智能体列表

### 基础示范

In [1]:
from illufly.chat import ChatQwen, ChatZhipu, FakeLLM, Selector
from illufly.io import alog

# 默认选择了第一个 Runnable 作为 selected
agent = Selector([FakeLLM(["abc", "df"]), ChatQwen(), ChatZhipu()], condition="first")
agent.select()
agent.bind_provider({"task": "写一首歌"})

agent("你是什么模型？", verbose=True)

[INFO] 记住 10 轮对话
[INFO] I am FakeLLM
abc


'abc'

In [2]:
agent.selected("你是什么模型？", verbose=True)

[INFO] 记住 10 轮对话
[INFO] I am FakeLLM
df


'df'

In [3]:
agent.selected

<FakeLLM.4504487392>

### 路由 ChatAgent

考虑到你有两个专业的 ChatAgent:

- data_agent 擅长数据分析
- writer_agent 擅长写作

你已经根据为他们分别配置了提示语模板、工具集、数据库等，但现在你需要根据用户的意图匹配到底使用哪一个智能体。<br>

那么如何判断用户意图呢？
- 方案1 可以使用大模型来做甄别，这会增加一个环节，推理过程可能延长用户的等待
- 方案2 可以根据问题做文本相似性比较，这样做效率很高

我们的例子中探讨方案2如何实现，关键就是与什么做比较？

原始的可比较信息是：用户问题与智能体描述文本之间的比较，但这明显不太够用。<br>
可以使用**检索器**和**经验缓存**来弥补。


In [3]:
from illufly.chat import ChatQwen, ChatZhipu, FakeLLM, Selector
from illufly.embeddings import TextEmbeddings

agent = Selector(
    [
        FakeLLM(description="模拟调用"),
        ChatQwen(description="写歌"),
        ChatZhipu(description="数据分析")
    ],
    condition="similar",
    embeddings=TextEmbeddings()
)

task = "写一首歌，3句"
agent.bind_provider({"task": task})
agent(task)

在每个黎明破晓前，我与希望并肩站，
穿越风雨拥抱晴天，你是我心中的光，
这首歌献给梦，献给爱，献给每一个不屈的瞬间。


'在每个黎明破晓前，我与希望并肩站，\n穿越风雨拥抱晴天，你是我心中的光，\n这首歌献给梦，献给爱，献给每一个不屈的瞬间。'

In [3]:
agent.consumer_dict

{'task': '写一首歌，3句'}

### 路由 PromptTemplate

In [2]:
from illufly.types import PromptTemplate
from illufly.chat import Selector

r = Selector(
    runnables=[PromptTemplate(text="请帮我：{{task}}"), PromptTemplate(text="你是一个作家")]
)
r.bind_provider({"task": "写一首歌"})

r.selected

<PromptTemplate consumer_dict={'task'} text='请帮我：{{task}}'>

使用 lazy_binding_map 可以先声明对象，然后在合适的时机自动实现绑定。

In [1]:
from illufly.chat import ChatQwen, Selector
from illufly.types import PromptTemplate

t = Selector(
    [
        PromptTemplate(text="请帮我写一首关于《{{task}}》的儿歌，四句"),
        PromptTemplate(text="请帮我起一个{{name}}的名字，3个字", lazy_binding_map={"name": "task"})
    ],
    condition=lambda vars, runs: runs[0]
)
qwen = ChatQwen(memory=[t])
qwen("小兔子")

小兔子乖乖，耳朵竖起来，
蹦蹦跳跳真可爱，红红眼睛像玫瑰。 

注：一般儿歌为了便于儿童记忆和朗诵，通常会采用简单重复的结构，这里第三、第四句稍微变化，强调了小兔子的活泼与美丽特征。实际上，可以根据需要进一步简化，保持重复性，如：“小兔子乖乖，耳朵竖起来，蹦蹦跳跳真可爱，乖乖小兔子乖。” 这样的方式更有助于孩子们学习和记忆。


'小兔子乖乖，耳朵竖起来，\n蹦蹦跳跳真可爱，红红眼睛像玫瑰。 \n\n注：一般儿歌为了便于儿童记忆和朗诵，通常会采用简单重复的结构，这里第三、第四句稍微变化，强调了小兔子的活泼与美丽特征。实际上，可以根据需要进一步简化，保持重复性，如：“小兔子乖乖，耳朵竖起来，蹦蹦跳跳真可爱，乖乖小兔子乖。” 这样的方式更有助于孩子们学习和记忆。'

## 自定义 Selector

### 返回 Runnable

In [1]:
from illufly.chat import ChatQwen, Selector
from illufly.types import PromptTemplate

t = Selector(
    [
        PromptTemplate(text="请帮我写一首关于《{{task}}》的儿歌，四句"),
        PromptTemplate(text="请帮我起一个{{name}}的名字，3个字", lazy_binding_map={"name": "task"})
    ],
    condition=lambda vars, runs: runs[0]
)
t.selected

<PromptTemplate consumer_dict={'task'} text='请帮我写一首关于《{{task}}》的儿歌，四句'>

### 返回 name

In [2]:
from illufly.chat import ChatQwen, Selector
from illufly.types import PromptTemplate

t = Selector(
    [
        PromptTemplate(text="请帮我写一首关于《{{task}}》的儿歌，四句"),
        PromptTemplate(text="请帮我起一个{{name}}的名字，3个字", lazy_binding_map={"name": "task"})
    ],
    condition=lambda vars, runs: runs[0].name
)
t.selected

'PromptTemplate.4365337936'

### 返回 End

In [4]:
from illufly.chat import ChatQwen, Selector
from illufly.types import PromptTemplate

t = Selector(
    [
        PromptTemplate(text="请帮我写一首关于《{{task}}》的儿歌，四句"),
        PromptTemplate(text="请帮我起一个{{name}}的名字，3个字", lazy_binding_map={"name": "task"})
    ],
    condition=lambda: "End"
)
t.selected

<__End__>